<a href="https://colab.research.google.com/github/igornunespatricio/BairesDev-Machine-Learning-Practitioner/blob/main/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Recognition

Steps

1) Upload raw images in directory uploaded_folder

2) Detect faces using MTCNN and store in the faces directory

3) Use pre-trained model InceptionResnetV1 to extract embeddings from faces in the faces directory

4) Use DBSCAN to cluster the embeddings and generate the organized_faces directory automatically with a folder for each person

5) Encoding known faces: generate a pickle file with known face embeddings

6) Recognize faces: upload unseen images and compare detected faces with known embeddings.

In [1]:
!pip install fastcore -U
!pip install Pillow==9.3.0 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: fastcore
    Found existing installation: fastcore 1.7.20
    Uninstalling fastcore-1.7.20:
      Successfully uninstalled fastcore-1.7.20
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0


In [1]:
!pip install tensorflow opencv-python mtcnn keras-facenet facenet-pytorch --quiet
!pip install face_recognition numpy --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━

## 1) Upload raw images from local

Ask user to upload raw images from local to colab and store it in the uploaded_images directory.

In [2]:
import os
from google.colab import files

# Create the directory if it doesn't exist
upload_dir = 'uploaded_images'
if not os.path.exists(upload_dir):
    os.makedirs(upload_dir)

# Upload images
uploaded = files.upload()

print("\nFiles uploaded successfully.")

# Move uploaded images to the designated directory
for filename in uploaded.keys():
    os.rename(filename, os.path.join(upload_dir, filename))

print(f"\nUploaded files saved to '{upload_dir}' directory.")


Saving WhatsApp Image 2024-11-30 at 15.35.39 (2).jpeg to WhatsApp Image 2024-11-30 at 15.35.39 (2).jpeg
Saving WhatsApp Image 2024-11-30 at 15.35.39 (1).jpeg to WhatsApp Image 2024-11-30 at 15.35.39 (1).jpeg
Saving WhatsApp Image 2024-11-30 at 15.35.39.jpeg to WhatsApp Image 2024-11-30 at 15.35.39.jpeg
Saving WhatsApp Image 2024-11-30 at 15.35.38 (1).jpeg to WhatsApp Image 2024-11-30 at 15.35.38 (1).jpeg
Saving WhatsApp Image 2024-11-30 at 15.35.38.jpeg to WhatsApp Image 2024-11-30 at 15.35.38.jpeg
Saving WhatsApp Image 2024-11-30 at 15.33.35.jpeg to WhatsApp Image 2024-11-30 at 15.33.35.jpeg
Saving WhatsApp Image 2024-11-30 at 15.11.29 (3).jpeg to WhatsApp Image 2024-11-30 at 15.11.29 (3).jpeg
Saving WhatsApp Image 2024-11-30 at 15.11.29 (2).jpeg to WhatsApp Image 2024-11-30 at 15.11.29 (2).jpeg
Saving WhatsApp Image 2024-11-30 at 15.11.29 (1).jpeg to WhatsApp Image 2024-11-30 at 15.11.29 (1).jpeg
Saving WhatsApp Image 2024-11-30 at 15.11.29.jpeg to WhatsApp Image 2024-11-30 at 15.11.

In [ ]:
import cv2
import os
from mtcnn import MTCNN

## 2) MTCNN model to detect faces

Run MTCNN pre trained model to detect faces on the images of the uploaded_images directory and store the faces in the faces directory.

In [4]:
# Initialize the MTCNN face detector
detector = MTCNN()

# Path to the uploaded images
image_folder = '/content/uploaded_images'

# Create a folder to save cropped faces
os.makedirs('/content/faces', exist_ok=True)

# Loop through the images in the folder
for filename in os.listdir(image_folder):
    img_path = os.path.join(image_folder, filename)

    # Read the image
    img = cv2.imread(img_path)

    # Convert the image to RGB (OpenCV loads images in BGR)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    faces = detector.detect_faces(img_rgb)

    # Crop faces and save them
    for i, face in enumerate(faces):
        x, y, w, h = face['box']
        face_img = img[y:y+h, x:x+w]
        cropped_face_path = f'/content/faces/{filename}_face_{i}.jpg'
        cv2.imwrite(cropped_face_path, face_img)


## 3) InceptionResnetV1 to extract embeddings

Use of pre trained InceptionResnetV1 model to extract embeddings from the detected faces in the faces directory.

In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
from facenet_pytorch import InceptionResnetV1
import torch

# Path to the folder containing the face images
face_folder = '/content/faces'

# Load the pre-trained InceptionResnetV1 model for face embedding extraction
inception = InceptionResnetV1(pretrained='vggface2').eval()

# Function to extract embeddings from the face image
def get_face_embedding(face_image):
    # Convert the image from BGR (OpenCV) to RGB
    face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)

    # Resize the image if necessary (standardize size for embedding generation)
    face_image_rgb = cv2.resize(face_image_rgb, (160, 160))  # Resize to 160x160 for the model

    # Convert the image to a tensor
    face_tensor = torch.tensor(face_image_rgb).permute(2, 0, 1).float() / 255.0  # Normalize to [0, 1] range
    face_tensor = face_tensor.unsqueeze(0)  # Add batch dimension

    # Get the face embedding
    embedding = inception(face_tensor)
    return embedding.detach().numpy()  # Convert tensor to numpy array

# List to store face images and embeddings
face_embeddings = []

# Loop through the images in the 'faces' folder and generate embeddings
for filename in os.listdir(face_folder):
    img_path = os.path.join(face_folder, filename)

    # Read the image
    img = cv2.imread(img_path)
    if img is not None:
        # Get the embedding for the detected face
        embedding = get_face_embedding(img)

        # Only store the embedding if it was successfully extracted
        if embedding is not None:
            face_embeddings.append(embedding)  # Store the embedding

# Convert embeddings list to numpy array for clustering
face_embeddings_np = np.array(face_embeddings)

# Reshape the embeddings array to 2D
# The embeddings are currently in shape (n_samples, 1, embedding_size)
# We need to reshape it to (n_samples, embedding_size) for DBSCAN
face_embeddings_np = face_embeddings_np.reshape(face_embeddings_np.shape[0], -1)

## 4) DBSCAN to cluster embeddings

Use DBSCAN to cluster similar faces according to their embeddings and automatically generate a folder for each person.

In [ ]:
# Perform DBSCAN clustering
eps = 1  # Maximum distance between points in the same cluster
min_samples = 2  # Minimum number of points to form a dense cluster
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(face_embeddings_np)


# Create a folder to save the clustered faces
clustered_folder = '/content/organized_faces'

# Create a folder for noise points (outliers)
noise_folder = os.path.join(clustered_folder, 'noise')
os.makedirs(noise_folder, exist_ok=True)

# Save faces into folders based on their cluster labels
for label in set(labels):
    if label == -1:
        # DBSCAN assigns -1 to noise points (outliers), save these faces in the 'noise' folder
        for i, label_i in enumerate(labels):
            if label_i == -1:
                # Construct the filename for the face image
                face_filename = f'face_{i}.jpg'
                face_path = os.path.join(noise_folder, face_filename)
                cv2.imwrite(face_path, cv2.imread(os.path.join(face_folder, os.listdir(face_folder)[i])))  # Save the noise face
        continue

    # Create a folder for each cluster
    cluster_folder = os.path.join(clustered_folder, f'person_{label}')
    os.makedirs(cluster_folder, exist_ok=True)

    # Loop through the images and move them to the corresponding cluster folder
    for i, label_i in enumerate(labels):
        if label_i == label:
            # Construct the filename for the face image
            face_filename = f'face_{i}.jpg'
            face_path = os.path.join(cluster_folder, face_filename)
            cv2.imwrite(face_path, cv2.imread(os.path.join(face_folder, os.listdir(face_folder)[i])))  # Save the face image

print("Faces organized into clusters, and noise faces are saved separately.")

In [7]:
from collections import Counter

count = Counter(labels)

# Print the number of faces in each class
for class_value, num_faces in count.items():
    print(f"Number of faces in class {class_value}: {num_faces}")

Number of faces in class 0: 21
Number of faces in class 1: 16
Number of faces in class 2: 3
Number of faces in class -1: 1


## 5) Encoding known faces

Take the faces and encode them in the embedding.

In [8]:
import pickle

def encode_faces(dataset_path):
    known_face_encodings = []
    known_face_names = []

    for person_name in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person_name)
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            img = cv2.imread(image_path)
            if img is not None:
                embedding = get_face_embedding(img)
                if embedding is not None:
                    known_face_encodings.append(embedding)
                    known_face_names.append(person_name)

    # Save encodings and names to a pickle file
    with open("known_faces.pkl", "wb") as f:
        pickle.dump((known_face_encodings, known_face_names), f)

# Run the encoding function
encode_faces("/content/organized_faces")


## 6) Recognize faces

In [9]:
import os
from google.colab import files

# Create the directory if it doesn't exist
upload_dir = 'predict-these-images'
if not os.path.exists(upload_dir):
    os.makedirs(upload_dir)

# Upload images
uploaded = files.upload()

print("\nFiles uploaded successfully.")

# Move uploaded images to the designated directory
for filename in uploaded.keys():
    os.rename(filename, os.path.join(upload_dir, filename))

print(f"\nUploaded files saved to '{upload_dir}' directory.")


Saving WhatsApp Image 2024-12-02 at 20.44.18.jpeg to WhatsApp Image 2024-12-02 at 20.44.18.jpeg
Saving WhatsApp Image 2024-12-02 at 20.44.19.jpeg to WhatsApp Image 2024-12-02 at 20.44.19.jpeg

Files uploaded successfully.

Uploaded files saved to 'predict-these-images' directory.


In [12]:
from google.colab.patches import cv2_imshow  # If using Google Colab

# Load known face embeddings and names
with open('/content/known_faces.pkl', 'rb') as f:
    known_face_encodings, known_face_names = pickle.load(f)

known_face_encodings = np.array(known_face_encodings).squeeze(axis=1)  # Now shape is (41, 512)

# Initialize face detector
detector = MTCNN()

# Path to the directory containing images to predict
directory = '/content/predict-these-images'

results_path = '/content/results'

os.makedirs(results_path, exist_ok=True)

# Loop through all images in the directory
for filename in os.listdir(directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Process only image files
        image_path = os.path.join(directory, filename)
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Detect faces in the image
        faces = detector.detect_faces(img_rgb)

        for face in faces:
            x, y, w, h = face['box']
            face_img = img[y:y+h, x:x+w]

            # Generate embedding for the detected face
            unknown_embedding = get_face_embedding(face_img)

            if unknown_embedding is not None:
                unknown_embedding = np.array(unknown_embedding).squeeze(axis=0)  # Now shape is (512,)
                distances = np.linalg.norm(known_face_encodings - unknown_embedding, axis=1)

                # Find the best match
                min_distance = np.min(distances)
                min_distance_index = np.argmin(distances)

                # Set a threshold for recognition
                threshold = 1.0  # Adjust this value as needed

                if min_distance <= threshold:
                    name = known_face_names[min_distance_index]
                    print(f"Recognized: {name} in {filename} (Distance: {min_distance})")
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(img, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                else:
                    print(f"Unknown face in {filename}")
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(img, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Display or save the image with bounding boxes and labels
        # cv2_imshow(img)  # If using Google Colab
        cv2.imwrite(f'{results_path}/{filename}', img)  # Save the result with the same filename in a 'results' folder


Recognized: person_0 in WhatsApp Image 2024-12-02 at 20.44.19.jpeg (Distance: 0.7033995389938354)
Recognized: person_1 in WhatsApp Image 2024-12-02 at 20.44.19.jpeg (Distance: 0.7914150357246399)
Recognized: person_0 in WhatsApp Image 2024-12-02 at 20.44.18.jpeg (Distance: 0.4772179424762726)
Recognized: person_1 in WhatsApp Image 2024-12-02 at 20.44.18.jpeg (Distance: 0.6739935278892517)
Recognized: person_2 in WhatsApp Image 2024-12-02 at 20.44.18.jpeg (Distance: 0.4197559952735901)
